# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
import os
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data/'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism'

# upload all data to S3
train_path = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)
test_path = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
print(train_path)
print(test_path)

s3://sagemaker-us-west-2-480263271248/plagiarism/train.csv
s3://sagemaker-us-west-2-480263271248/plagiarism/test.csv


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

energy_ts/forecasting-deepar-2020-05-16-10-18-00-416/output/model.tar.gz
energy_ts/test.json
energy_ts/train.json
plagiarism/sagemaker-pytorch-200525-0925-001-4687a96a/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-002-0a4d8864/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-003-bfb546dc/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-004-7a892b33/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-005-d9d892c5/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-006-1f4bc623/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-007-09d37b48/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-008-659c04a7/output/model.tar.gz
plagiarism/sagemaker-pytorch-200525-0925-009-ff103f69/output/model.tar.gz
plagiarism/sagemaker-scikit-learn-2020-05-22-11-33-46-445/debug-output/training_job_end.ts
plagiarism/sagemaker-scikit-learn-2020-05-22-11-33-46-445/output/model.tar.gz
plagiarism/sagemaker-scikit-learn-2020-05-22-12-35-

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [5]:
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.pytorch import PyTorch

In [6]:
svc = SKLearn(entry_point='train_svc.py',
             source_dir='source_sklearn',
             role=role,
             train_instance_count=1,
             train_instance_type='ml.m4.xlarge',
             output_path='s3://{}/{}/'.format(sagemaker_session.default_bucket(), prefix),
             hyperparameters={})

In [7]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train_svc.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model

if __name__ == '__main__':
    # SageMaker parameters, like the directories for training data and saving models; set automatically
    parser = argparse.ArgumentParser()
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.

In [8]:
gpc = SKLearn(entry_point='train_gpc.py',
             source_dir='source_sklearn',
             role=role,
             train_instance_count=1,
             train_instance_type='ml.m4.xlarge',
             output_path='s3://{}/{}/'.format(sagemaker_session.default_bucket(), prefix))

In [9]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train_gpc.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import GridSearchCV



# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model

if __name__ == '__main__':
    # SageMaker parameters, like the directories for training data and saving models
    parser = argparse.ArgumentParser()
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('-

In [10]:
# NOTE: naive bayes classifiers assume data features are not highly correlated, which is not the case with
## our data. We will be calculating the NBC just to see its performance on highly correlated data.

nbc = SKLearn(entry_point='train_nbc.py',
             source_dir='source_sklearn',
             role=role,
             train_instance_count=1,
             train_instance_type='ml.m4.xlarge',
             output_path='s3://{}/{}/'.format(sagemaker_session.default_bucket(), prefix))

In [11]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train_nbc.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB as NB


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model

if __name__ == '__main__':
    # SageMaker parameters, like the directories for training data and saving models
    parser = argparse.ArgumentParser()
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data-dir', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    
    

In [12]:
neuralnet = PyTorch(entry_point='train.py',
                   source_dir='source_pytorch',
                   role=role,
                   train_instance_count=1,
                   train_instance_type='ml.p2.xlarge',
                   framework_version='1.0',
                   output_path='s3://{}/{}/'.format(sagemaker_session.default_bucket(), prefix),
                   hyperparameters={'epochs': 100,
                                   'lr': 0.01,
                                   'batch_size': 10,
                                   'input_features': 3,
                                   'hidden_dim': 32,
                                   'output_dim': 1})

In [13]:
!pygmentize source_pytorch/train.py

import argparse
import json
import logging
import os
import pandas as pd
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import BinaryClassifier

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    logger.info("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    logger.info("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model par

In [14]:
!pygmentize source_pytorch/model.py

# torch imports
import torch.nn.functional as F
import torch.nn as nn


class BinaryClassifier(nn.Module):
    """
    Defines a neural network that performs binary classification.
    """
    def __init__(self, input_features, hidden_dim, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: helps define the number of nodes in the hidden layer(s)
        :param output_dim: the number of outputs you want to produce
        """
        super(BinaryClassifier, self).__init__()
        self.model = nn.Sequential(nn.Linear(input_features, hidden_dim),
                                  nn.ReLU(),
                                  nn.Dropout(0.4),
                                  nn.Linear(hidden_dim, hidden_dim),
                                  nn.ReLU(),
                   

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [15]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
s3_input_train = sagemaker.s3_input(s3_data=train_path, content_type='text/csv')

In [16]:
%%time

# Train your estimator on S3 training data
svc.fit({'train': s3_input_train})

2020-05-25 10:22:48 Starting - Starting the training job...
2020-05-25 10:22:50 Starting - Launching requested ML instances.........
2020-05-25 10:24:24 Starting - Preparing the instances for training...
2020-05-25 10:25:15 Downloading - Downloading input data...
2020-05-25 10:25:47 Training - Downloading the training image...
2020-05-25 10:26:19 Uploading - Uploading generated training model
2020-05-25 10:26:19 Completed - Training job completed
2020-05-25 10:26:07,585 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-25 10:26:07,588 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-25 10:26:07,599 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-25 10:26:07,868 sagemaker-containers INFO     Module train_svc does not provide a setup.py. 
Generating setup.py
2020-05-25 10:26:07,868 sagemaker-containers INFO     Generating setup.cfg
2020-05-25 10:26:07,868 sagemaker-contain

In [17]:
%%time

# Train your estimator on S3 training data
gpc.fit({'train': s3_input_train})

2020-05-25 10:26:30 Starting - Starting the training job...
2020-05-25 10:26:32 Starting - Launching requested ML instances......
2020-05-25 10:27:39 Starting - Preparing the instances for training......
2020-05-25 10:28:39 Downloading - Downloading input data...
2020-05-25 10:29:26 Training - Training image download completed. Training in progress..2020-05-25 10:29:27,173 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-25 10:29:27,176 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-25 10:29:27,188 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-25 10:29:27,494 sagemaker-containers INFO     Module train_gpc does not provide a setup.py. 
Generating setup.py
2020-05-25 10:29:27,495 sagemaker-containers INFO     Generating setup.cfg
2020-05-25 10:29:27,495 sagemaker-containers INFO     Generating MANIFEST.in
2020-05-25 10:29:27,495 sagemaker-containers INFO     Installin

In [18]:
%%time

# Train your estimator on S3 training data
nbc.fit({'train': s3_input_train})

2020-05-25 10:30:12 Starting - Starting the training job...
2020-05-25 10:30:14 Starting - Launching requested ML instances.........
2020-05-25 10:31:50 Starting - Preparing the instances for training......
2020-05-25 10:32:46 Downloading - Downloading input data...
2020-05-25 10:33:40 Training - Training image download completed. Training in progress..2020-05-25 10:33:41,292 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-05-25 10:33:41,294 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-05-25 10:33:41,307 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-05-25 10:33:56,986 sagemaker-containers INFO     Module train_nbc does not provide a setup.py. 
Generating setup.py
2020-05-25 10:33:56,986 sagemaker-containers INFO     Generating setup.cfg
2020-05-25 10:33:56,986 sagemaker-containers INFO     Generating MANIFEST.in
2020-05-25 10:33:56,986 sagemaker-containers INFO     Instal

In [19]:
%%time

# Train your estimator on S3 training data
hyperparameter_ranges = {'batch_size': CategoricalParameter([6, 10, 12, 16, 32]),
                        'epochs': IntegerParameter(10,20),
                        'lr': ContinuousParameter(0.001, 0.1),
                        'hidden_dim': CategoricalParameter([10, 16, 32, 64, 96])}
objective_metric_name = 'BCE Loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'BCE Loss',
                      'Regex': "Binary Cross-Entropy Loss: ([0-9\\.]+)"}]

nn_tuner = HyperparameterTuner(estimator=neuralnet,
                              objective_metric_name=objective_metric_name,
                              objective_type=objective_type,
                              metric_definitions=metric_definitions,
                              max_jobs=9,
                              max_parallel_jobs=3,
                              hyperparameter_ranges=hyperparameter_ranges)

nn_tuner.fit({'train': s3_input_train})
nn_tuner.wait()

..............................................................................................................................................................................!
CPU times: user 930 ms, sys: 66 ms, total: 996 ms
Wall time: 14min 39s


In [21]:
from sagemaker.pytorch import PyTorchModel

nn_predictor = PyTorchModel(model_data=nn_tuner.best_estimator().model_data,
                           role=role,
                           entry_point='predict.py',
                           source_dir='source_pytorch',
                           framework_version='1.0')

2020-05-25 10:47:44 Starting - Preparing the instances for training
2020-05-25 10:47:44 Downloading - Downloading input data
2020-05-25 10:47:44 Training - Training image download completed. Training in progress.
2020-05-25 10:47:44 Uploading - Uploading generated training model
2020-05-25 10:47:44 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-05-25 10:47:28,075 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-05-25 10:47:28,076 sagemaker-containers INFO     Failed to parse hyperparameter _tuning_objective_metric value BCE Loss to Json.
Returning the value itself
2020-05-25 10:47:28,101 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-05-25 10:47:28,102 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-05-25 10:47:28,331 sagemaker-containers INFO     Module train doe

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [22]:
%%time
# 

# deploy your model to create a predictor
svc_predictor = svc.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
gpc_predictor = gpc.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
nbc_predictor = nbc.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
nn_predictor = nn_predictor.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------!-----------------!---------------!---------------!CPU times: user 1.39 s, sys: 43.5 ms, total: 1.43 s
Wall time: 31min 11s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [24]:
# First: generate predicted, class labels
test_svc_preds = svc_predictor.predict(test_x)
test_gpc_preds = gpc_predictor.predict(test_x)
test_nbc_preds = nbc_predictor.predict(test_x)
test_nn_preds = nn_predictor.predict(test_x)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_svc_preds)==len(test_y), 'Unexpected number of predictions.'
print('SVC test passed!')
assert len(test_gpc_preds)==len(test_y), 'Unexpected number of predictions.'
print('GPC test passed!')
assert len(test_nbc_preds)==len(test_y), 'Unexpected number of predictions.'
print('NBC test passed!')
assert len(test_nn_preds)==len(test_y), 'Unexpected number of predictions.'
print('Neural Net test passed!')

SVC test passed!
GPC test passed!
NBC test passed!
Neural Net test passed!


In [25]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score

svc_accuracy = accuracy_score(test_y, test_svc_preds)
svc_precision = precision_score(test_y, test_svc_preds)
svc_recall = recall_score(test_y, test_svc_preds)
print('\n\nSVC PREDICTOR:')
print('------------------------------------')
print('Accuracy: ', svc_accuracy)
print('Precision: ', svc_precision)
print('Recall: ', svc_recall)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_svc_preds)
print('\nTrue class labels: ')
print(test_y.values)


gpc_accuracy = accuracy_score(test_y, test_gpc_preds)
gpc_precision = precision_score(test_y, test_gpc_preds)
gpc_recall = recall_score(test_y, test_gpc_preds)
print('\n\nGPC PREDICTOR:')
print('------------------------------------')
print('Accuracy: ', gpc_accuracy)
print('Precision: ', gpc_precision)
print('Recall: ', gpc_recall)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_gpc_preds)
print('\nTrue class labels: ')
print(test_y.values)


nbc_accuracy = accuracy_score(test_y, test_nbc_preds)
nbc_precision = precision_score(test_y, test_nbc_preds)
nbc_recall = recall_score(test_y, test_nbc_preds)
print('\n\nNBC PREDICTOR:')
print('------------------------------------')
print('Accuracy: ', nbc_accuracy)
print('Precision: ', nbc_precision)
print('Recall: ', nbc_recall)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_nbc_preds)
print('\nTrue class labels: ')
print(test_y.values)


nn_accuracy = accuracy_score(test_y, test_nn_preds)
nn_precision = precision_score(test_y, test_nn_preds)
nn_recall = recall_score(test_y, test_nn_preds)
print('\n\nNEURAL NET PREDICTOR:')
print('------------------------------------')
print('Accuracy: ', nn_accuracy)
print('Precision: ', nn_precision)
print('Recall: ', nn_recall)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_nn_preds)
print('\nTrue class labels: ')
print(test_y.values)



SVC PREDICTOR:
------------------------------------
Accuracy:  1.0
Precision:  1.0
Recall:  1.0

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]


GPC PREDICTOR:
------------------------------------
Accuracy:  1.0
Precision:  1.0
Recall:  1.0

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]


NBC PREDICTOR:
------------------------------------
Accuracy:  0.92
Precision:  0.8823529411764706
Recall:  1.0

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 1.]

True class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]


NEURAL NET PREDICTOR:
------------------------------------
Accuracy:  1.0
Preci

### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**: 


The Support Vector Classifier, Gaussian Process Classifiar and Neural Net Classifier didn't produce any false positives or false negatives. They all have accuracy, precision and recall scores of 1.0. The Naive Baye's Classifier did produce two false positives, due to the high correlation of the input features. Naive Baye's assumes input data features are not significantly correlated. However, I would argue that, the most important metric for a plagiarism detector in production would be having a high recall, i.e. finding the minimum number of false negatives possible. All positives would probably be flagged for a human in the loop to double check them. False negatives would be a real issue, since that would mean no human checking for plagiarism on a plagiarized text. Naive Baye's obatins a 1.0 recall score for this simple dataset, which would still be acceptable.

### Question 2: How did you decide on the type of model to use? 

**Answer**:


I chose to try different classification models and tune each model's hyperparameters to see the difference in performance for different possible algorithms, as well as different hyperparameter combinations.

**SKLearn** models perform hyperparameter tuning within the train.py script. They perform a grid search over the discrete possible combination of hyperparameters, with the scikit-learn algorithm GridSearchCV.

It is interesting to note that the tuning loop found C=100, kernel=linear as its best parameters for the **SVC**. This means that the data distribution is simple enough to be separated by linear classifiers, and radius basis function kernels, although still successful underperform them. Additionally, it chooses the highest possible C I allowed the model to have (100.0) since the tuning job has minimizing the classification error objetive metric as a goal. Thus, a higher C gives greater importance to misclassification in the error function.

The **Gaussian Process Classifier** tuning job alternated between isotropic and anisotropic kernels. And the **Naive Baye's Classifier** tunes its var smoothing hyperparameter, although it doesn't have a very significant impact. I chose to include a Naive Baye's classifier to see how breaking the correlation assumption of the model impacted its performance relative to the other models.

Additionally, I created a linear neural net classifier in PyTorch with three layers, ReLU activations, and dropout layers in the 2 first layers. The model found a combination of 10 epochs, hidden dim width of 96 neurons, learning rate of 0.0083 and batch size of 10, to be optimal.

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [26]:
svc_predictor.delete_endpoint()
gpc_predictor.delete_endpoint()
nbc_predictor.delete_endpoint()
nn_predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [ ]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!